# Get Predictions from [AIforThai (xiaofan)](https://aiforthai.in.th/aiplatform/#/chinesetothai)

Prediction results with AIforThai xiaofan run on 1,500 randomly selected sentence pairs of the test set, due to limitations of the AIforThai API, on 2021-06-21 done by [@wannaphong](http://python3.wannaphong.com/)

In [2]:
import pandas as pd
import requests
from urllib.parse import quote_plus

In [2]:
headers = {
    'Apikey': "", # It is my cat.
    }

In [3]:
import sqlite3

In [4]:
conn = sqlite3.connect('ch2.db')
print("เปิดฐานข้อมูลสำเร็จ")
conn.execute('''CREATE TABLE zhth
       (ID INTEGER PRIMARY KEY   AUTOINCREMENT,
       DFID INT NOT NULL,
       ZH           TEXT    NOT NULL,
       TH        TEXT    NOT NULL);''')
conn.execute('''CREATE TABLE thzh
       (ID INTEGER PRIMARY KEY  AUTOINCREMENT,
       DFID INT NOT NULL,
       TH           TEXT    NOT NULL,
       ZH        TEXT    NOT NULL);''')
conn.commit()
print("สร้างตารางสำเร็จ :D ")
conn.close() 

เปิดฐานข้อมูลสำเร็จ
สร้างตารางสำเร็จ :D 


In [6]:
df = pd.read_csv('test1500_seed42.csv')

In [7]:
df

,th,zh_cn
0,และเพราะเหตุนี้ ฉันก็ต้องขอบอกว่า การร่วมมือกั...,而正因为这点，我必须得说， 合作是我最喜欢的超凡能力。
1,ถ้างานของคุณไม่เข้าขั้น ก็ไม่ใช่ความผิดของคุณท...,如果你的作品很烂，同样也不全是你的错
2,(เสียงหัวเราะ) (เสียงปรบมือ) ผมกำลังพูดถึงการท...,（笑）（掌声） 我所讲的就是让人们得到工作， 让孩子们远离街道，让他们了解 自己种植食物的喜...
3,เขากล่าวว่า ถ้าคุณอยู่ในโรงพยาบาล โรงพยาบาลจะส...,他写道：如果你去看病 医院对你的唯一帮助 就是提供了 给你提供了温暖、食物、住所， 或许还有...
4,และนี้คือบทเรียนที่ยิ่งใหญ่ที่สุด เรื่องสื่อสั...,接下来，就是我们在使用 社交媒体和手机时最大的教训：
...,...,...
1495,"ส่วนใหญ่นำเสนอในรูปของการ์ตูนช่องเดียว ""ฉันปรา...",他们通常会以单片式动画形式呈现， “我要谴责恐怖主义！同时我也谴责 那些从来不把纸盘子装满的...
1496,เทมเปิล: ฉันรักสิ่งที่ฉันทำ อยากให้สิ่งฉันทำ เ...,葛兰丁：我对于我所做的事情感到十分热情 并且希望这些事情可以让世界变得更美好 当我听到有母亲说
1497,ฉันหวังว่าวันนี้ คุณจะมีมุมมองต่อป่า ที่ต่างออ...,我希望今天我也能改变 你们的一些关于森林的印象。
1498,ว่าจะเงาตกอยู่ตรงไหน ที่เป็นกลางคืน และตรงไหนม...,有阴影的地方是黑夜， 而有阳光的地方就是白天。


In [8]:
conn = sqlite3.connect('ch2.db', check_same_thread=False)

In [9]:
th2zhlist = []

In [10]:
def th2zh(text,id):
    url = 'https://api.aiforthai.in.th/th-zh-nmt/' +quote_plus(text)
    response = requests.get(url, headers=headers)
    if "The upstream server is timing out" in response.text:
        response = requests.get(url, headers=headers)
    conn.execute("INSERT INTO thzh (DFID,TH,ZH) VALUES (?, ?,?)",(id,text,response.text))
    conn.commit()

In [11]:
th2zh("ผมรักคุณ",-2)

In [12]:
#th2zh("แมว",0)

In [13]:
th2zhlist

[]

In [14]:
from concurrent.futures import ThreadPoolExecutor

In [15]:
from  tqdm import  tqdm

In [16]:
executor = ThreadPoolExecutor(max_workers=5)

In [17]:
th2zhlist = []
j=0
for i in tqdm(df['th'].tolist()):
    executor.submit(th2zh,i,j)
    j+=1

100%|██████████| 1500/1500 [00:00<00:00, 26143.38it/s]


In [18]:
len(th2zhlist)

0

In [19]:
zh2thlist = []

In [22]:
def zh2th(text,id):
    url = 'https://api.aiforthai.in.th/zh-th-nmt/' +quote_plus(text)
    response = requests.get(url, headers=headers)
    if "The upstream server is timing out" in response.text:
        response = requests.get(url, headers=headers)
    conn.execute("INSERT INTO zhth (DFID,ZH,TH) VALUES (?, ?,?)",(id,text,response.text))
    conn.commit()

In [41]:
def zh2th_reload(text,id):
    url = 'https://api.aiforthai.in.th/zh-th-nmt/' +quote_plus(text)
    response = requests.get(url, headers=headers)
    if "The upstream server is timing out" in response.text:
        response = requests.get(url, headers=headers)
    conn.execute("UPDATE zhth set TH=? WHERE id=?",(response.text,id))
    conn.commit()

In [48]:
def th2zh_reload(text,id):
    url = 'https://api.aiforthai.in.th/th-zh-nmt/' +quote_plus(text)
    response = requests.get(url, headers=headers)
    if "The upstream server is timing out" in response.text:
        response = requests.get(url, headers=headers)
    conn.execute("UPDATE thzh set ZH=? WHERE id=?",(response.text,id))
    conn.commit()

In [44]:
# zh2thlist = []
# jj=0
# for i in tqdm(df['zh_cn'].tolist()):
#     executor2.submit(zh2th,i,jj)
#     jj+=1

In [61]:
cursor = conn.execute("SELECT id,zh  from zhth WHERE th='The upstream server is timing out\n';")

In [62]:
for row in tqdm(cursor):
    #zh2th_reload(row[1],row[0])
    executor.submit(zh2th_reload,row[1],row[0])

6it [00:00, 159.47it/s]


In [58]:
cursor2 = conn.execute("SELECT id,th  from thzh WHERE zh='The upstream server is timing out\n';")

In [59]:
executor2 = ThreadPoolExecutor(max_workers=5)

In [60]:
for row in tqdm(cursor2):
    #print(row)
    executor2.submit(th2zh_reload,row[1],row[0])

185it [00:00, 4210.53it/s]


In [72]:
zh_th_cursor = conn.execute("SELECT DFID,zh,th  from zhth")
zh_th_tuples = []
for row in tqdm(zh_th_cursor):
    if row[0]>=0:
        zh_th_tuples.append(row)

1500it [00:00, 51140.90it/s]


In [73]:
zh_th_tuples[0]

(0,
 '而正因为这点，我必须得说， 合作是我最喜欢的超凡能力。 ',
 'สิ่งที่ผมจะต้องพูดก็คือมันเป็นสิ่งที่ผมชอบที่สุด')

In [74]:
zh_th_tuples[13]

(16,
 '如果确实存在危机， 如果这种危机是由 我们的排放造成的， 你至少该看到一些迹象。 ',
 'ถ้าเรามีวิกฤติการณ์บวกบวกบวกบวกบวกลบๆ')

In [75]:
zh_th_tuples = sorted(zh_th_tuples, key=lambda x: x[0])

In [76]:
zh_th_tuples[13]

(13, '在超过175个社会里，人们都用不同的证据证明了这部分大脑组织是如此强大 ', 'ในสังคม')

In [81]:
th_zh_cursor = conn.execute("SELECT DFID,zh,th from thzh")
th_zh_tuples = []
for row in tqdm(th_zh_cursor):
    if row[0]>=0:
        th_zh_tuples.append(row)

1501it [00:00, 189988.54it/s]


In [82]:
th_zh_tuples[0]

(1,
 '如果你的工作不深入,你就不是你的错。',
 'ถ้างานของคุณไม่เข้าขั้น ก็ไม่ใช่ความผิดของคุณทั้งหมด ')

In [83]:
len(th_zh_tuples)

1500

In [84]:
th_zh_tuples[13]

(15,
 '作为一个项目',
 'ในฐานะนักชีววิทยา ผมยังพยายามเรียนรู้ กับผู้อื่น ว่าทุก ๆ คนเกี่ยวข้องกันและกันอย่างไร และใครเกี่ยวข้องกับใคร ')

In [85]:
th_zh_tuples = sorted(th_zh_tuples, key=lambda x: x[0])

In [86]:
th_zh_tuples[13]

(13,
 '业余项目',
 'ในมากกว่า 175 สังคมวัฒนธรรม คนเราได้ทิ้งหลักฐาน ของระบบสมองอันทรงพลังนี้ ')

In [96]:
def check_error(txt):
    if 'The' in txt or 'the' in txt:
        return None
    else:
        return txt

In [97]:
dict_data = {
    'TH': [i[2] for i in th_zh_tuples],
    'ZH': [i[1] for i in zh_th_tuples],
    'pred_th': [check_error(i[2])  for i in zh_th_tuples],
    'pred_zh': [check_error(i[1])  for i in th_zh_tuples]
}

In [98]:
test_aiforthai=pd.DataFrame.from_dict(dict_data)

In [99]:
test_aiforthai

,TH,ZH,pred_th,pred_zh
0,และเพราะเหตุนี้ ฉันก็ต้องขอบอกว่า การร่วมมือกั...,而正因为这点，我必须得说， 合作是我最喜欢的超凡能力。,สิ่งที่ผมจะต้องพูดก็คือมันเป็นสิ่งที่ผมชอบที่สุด,我的项目
1,ถ้างานของคุณไม่เข้าขั้น ก็ไม่ใช่ความผิดของคุณท...,如果你的作品很烂，同样也不全是你的错,ถ้างานของคุณห่วยมากแต่ก็ไม่ใช่ความผิดของคุณ,"如果你的工作不深入,你就不是你的错。"
2,(เสียงหัวเราะ) (เสียงปรบมือ) ผมกำลังพูดถึงการท...,（笑）（掌声） 我所讲的就是让人们得到工作， 让孩子们远离街道，让他们了解 自己种植食物的喜...,(เสียงปรบมือ),好吧
3,เขากล่าวว่า ถ้าคุณอยู่ในโรงพยาบาล โรงพยาบาลจะส...,他写道：如果你去看病 医院对你的唯一帮助 就是提供了 给你提供了温暖、食物、住所， 或许还有...,คำพูดของเขากล่าวว่าถ้าคุณไปพบหมอกอล์สและถ้าคุณ...,他说
4,และนี้คือบทเรียนที่ยิ่งใหญ่ที่สุด เรื่องสื่อสั...,接下来，就是我们在使用 社交媒体和手机时最大的教训：,ขั้นตอนต่อไปคือบทเรียนของบวกและโทรศัพท์,这是一个伟大的项目
...,...,...,...,...
1495,"ส่วนใหญ่นำเสนอในรูปของการ์ตูนช่องเดียว ""ฉันปรา...",他们通常会以单片式动画形式呈现， “我要谴责恐怖主义！同时我也谴责 那些从来不把纸盘子装满的...,"พวกเขามักจะถือรูปแบบการวาดรูปแบบเดี่ยวหรือ""เบิ...","我给你看看""+""""。"
1496,เทมเปิล: ฉันรักสิ่งที่ฉันทำ อยากให้สิ่งฉันทำ เ...,葛兰丁：我对于我所做的事情感到十分热情 并且希望这些事情可以让世界变得更美好 当我听到有母亲说,ผมกระตือรือร้นและกระตือรือร้นในสิ่งที่ฉันทำ,T好吧
1497,ฉันหวังว่าวันนี้ คุณจะมีมุมมองต่อป่า ที่ต่างออ...,我希望今天我也能改变 你们的一些关于森林的印象。,ผมหวังว่าผมได้เปลี่ยนสิ่งที่คุณเห็นในวันนี้,我希望你再来一个新的项目吧
1498,ว่าจะเงาตกอยู่ตรงไหน ที่เป็นกลางคืน และตรงไหนม...,有阴影的地方是黑夜， 而有阳光的地方就是白天。,การมีเงามืดในที่มืดและมืดมัวแต่งแสงแดดคือกลางวัน,你知道吧。


In [100]:
test_aiforthai.to_csv('aiforthai.csv',index=False)

## Benchmark

In [12]:
df = pd.read_csv('../data/aiforthai.csv')
df.columns = ['th','zh','pred_th','pred_zh']
df.shape, df.dropna().shape

((1500, 4), (1226, 4))

In [13]:
#choose only those without NAs
df = df.dropna().reset_index(drop=True)
df

,th,zh,pred_th,pred_zh
0,และเพราะเหตุนี้ ฉันก็ต้องขอบอกว่า การร่วมมือกั...,而正因为这点，我必须得说， 合作是我最喜欢的超凡能力。,สิ่งที่ผมจะต้องพูดก็คือมันเป็นสิ่งที่ผมชอบที่สุด,我的项目
1,ถ้างานของคุณไม่เข้าขั้น ก็ไม่ใช่ความผิดของคุณท...,如果你的作品很烂，同样也不全是你的错,ถ้างานของคุณห่วยมากแต่ก็ไม่ใช่ความผิดของคุณ,"如果你的工作不深入,你就不是你的错。"
2,(เสียงหัวเราะ) (เสียงปรบมือ) ผมกำลังพูดถึงการท...,（笑）（掌声） 我所讲的就是让人们得到工作， 让孩子们远离街道，让他们了解 自己种植食物的喜...,(เสียงปรบมือ),好吧
3,เขากล่าวว่า ถ้าคุณอยู่ในโรงพยาบาล โรงพยาบาลจะส...,他写道：如果你去看病 医院对你的唯一帮助 就是提供了 给你提供了温暖、食物、住所， 或许还有...,คำพูดของเขากล่าวว่าถ้าคุณไปพบหมอกอล์สและถ้าคุณ...,他说
4,และนี้คือบทเรียนที่ยิ่งใหญ่ที่สุด เรื่องสื่อสั...,接下来，就是我们在使用 社交媒体和手机时最大的教训：,ขั้นตอนต่อไปคือบทเรียนของบวกและโทรศัพท์,这是一个伟大的项目
...,...,...,...,...
1221,"ส่วนใหญ่นำเสนอในรูปของการ์ตูนช่องเดียว ""ฉันปรา...",他们通常会以单片式动画形式呈现， “我要谴责恐怖主义！同时我也谴责 那些从来不把纸盘子装满的...,"พวกเขามักจะถือรูปแบบการวาดรูปแบบเดี่ยวหรือ""เบิ...","我给你看看""+""""。"
1222,เทมเปิล: ฉันรักสิ่งที่ฉันทำ อยากให้สิ่งฉันทำ เ...,葛兰丁：我对于我所做的事情感到十分热情 并且希望这些事情可以让世界变得更美好 当我听到有母亲说,ผมกระตือรือร้นและกระตือรือร้นในสิ่งที่ฉันทำ,T好吧
1223,ฉันหวังว่าวันนี้ คุณจะมีมุมมองต่อป่า ที่ต่างออ...,我希望今天我也能改变 你们的一些关于森林的印象。,ผมหวังว่าผมได้เปลี่ยนสิ่งที่คุณเห็นในวันนี้,我希望你再来一个新的项目吧
1224,ว่าจะเงาตกอยู่ตรงไหน ที่เป็นกลางคืน และตรงไหนม...,有阴影的地方是黑夜， 而有阳光的地方就是白天。,การมีเงามืดในที่มืดและมืดมัวแต่งแสงแดดคือกลางวัน,你知道吧。


In [26]:
df.to_csv('test1226.csv',index=False)

In [14]:
from datasets import load_metric
metric = load_metric("sacrebleu")

In [27]:
#benchmark zh->th
predictions = df.loc[:,'pred_th'].tolist()
references = [[i] for i in df.loc[:,'th'].tolist()]

In [28]:
predictions[:10]

['สิ่งที่ผมจะต้องพูดก็คือมันเป็นสิ่งที่ผมชอบที่สุด',
 'ถ้างานของคุณห่วยมากแต่ก็ไม่ใช่ความผิดของคุณ',
 '(เสียงปรบมือ)',
 'คำพูดของเขากล่าวว่าถ้าคุณไปพบหมอกอล์สและถ้าคุณไปพบหมอกอล์สและครูบาลได้ให้บริการคุณ',
 'ขั้นตอนต่อไปคือบทเรียนของบวกและโทรศัพท์',
 'เราก็ได้รู้ว่าเราต้องหาทางออกไป',
 'ชื่อของเขาคือ"บบ"ขอบเขตของทะเลทราย',
 'ผมไม่ได้บอกอะไรบางอย่างที่จะแสดงให้เห็นว่าเขามีความกระตือรือร้น',
 'การทำงานสามชั่วโมง',
 'อย่าลืมว่าพวกเขาไม่ได้เรียนภาษาอังกฤษ']

In [29]:
references[:10]

[['และเพราะเหตุนี้ ฉันก็ต้องขอบอกว่า การร่วมมือกันเป็น พลังยอดมนุษย์ที่ฉันโปรดปราน '],
 ['ถ้างานของคุณไม่เข้าขั้น ก็ไม่ใช่ความผิดของคุณทั้งหมด '],
 ['(เสียงหัวเราะ) (เสียงปรบมือ) ผมกำลังพูดถึงการทำให้คนมีงานทำ และดึงเด็กๆ ออกจากท้องถนน สอนให้เขารู้จัก ความสุข ความภาคภูมิใจ และความมีเกียรติ จากการปลูกอาหารกินเอง เปิดตลาดนัดเกษตรกร '],
 ['เขากล่าวว่า ถ้าคุณอยู่ในโรงพยาบาล โรงพยาบาลจะสำหรับคุณ เพราะเหตุผลเดียวคือ ให้ความอบอุ่น อาหาร ที่พัก และอาจจะมีการดูแลเอาใจใส่ จากพยาบาล '],
 ['และนี้คือบทเรียนที่ยิ่งใหญ่ที่สุด เรื่องสื่อสังคมและโทรศัพท์มือถือสำหรับเราทุกคน '],
 ['เรารู้ว่า เราต้องประดิษฐ์ แนวทางของเราเอง '],
 ['ซึ่งชื่อนั่นมีความหมายว่า ชายขอบของทะเลทราย '],
 ['และผมไม่ได้หมายถึงผู้เชี่ยวชาญที่จะมาบอกคุณว่า คุณกำลังมองดูอะไรอยู่เท่านั้นนะครับ แต่รวมถึงคนที่มีความหลงใหลในซอกมุมต่างๆในเอกภพ ผู้ที่สามารถแบ่งปันความสนใจของเขา และทำให้เอกภพเป็นที่ที่น่าไปท่องเที่ยวได้ '],
 ['ทำงานสามชั่วโมง '],
 ['อย่าลืมว่าพวกเขาไม่รู้ภาษาอังกฤษ ']]

In [16]:
%%time 
metric.compute(predictions=predictions,references=references,tokenize='th_syllable')

I0623 12:17:49.704079 140034238318400 metric.py:412] Removing /home/cstorm125/.cache/huggingface/metrics/sacrebleu/default/default_experiment-ae388a23-320e-45f6-9801-76902f008390-1-0.arrow


CPU times: user 1min 33s, sys: 1.86 s, total: 1min 35s
Wall time: 1min 46s


{'score': 6.74293684356434,
 'counts': [8035, 3691, 1876, 983],
 'totals': [16951, 15725, 14500, 13283],
 'precisions': [47.401333254675244,
  23.472178060413356,
  12.937931034482759,
  7.400436648347512],
 'bp': 0.3732351700044612,
 'sys_len': 16951,
 'ref_len': 33657}

In [17]:
%%time 
metric.compute(predictions=predictions,references=references,tokenize='th_word')

I0623 12:17:51.585505 140034238318400 metric.py:412] Removing /home/cstorm125/.cache/huggingface/metrics/sacrebleu/default/default_experiment-1c2b851b-7fcc-451c-8e1b-04003de3757e-1-0.arrow


CPU times: user 781 ms, sys: 15.6 ms, total: 797 ms
Wall time: 1.87 s


{'score': 4.269998725951353,
 'counts': [4873, 1655, 666, 278],
 'totals': [12544, 11318, 10098, 8899],
 'precisions': [38.84725765306123,
  14.622724863050008,
  6.595365418894831,
  3.123946510843915],
 'bp': 0.41053006104713324,
 'sys_len': 12544,
 'ref_len': 23712}

In [18]:
#benchmark zh->th
predictions = df.loc[:,'pred_zh'].tolist()
references = [[i] for i in df.loc[:,'zh'].tolist()]

In [20]:
%%time 
metric.compute(predictions=predictions,references=references,tokenize='zh')

I0623 12:19:02.490108 140034238318400 metric.py:412] Removing /home/cstorm125/.cache/huggingface/metrics/sacrebleu/default/default_experiment-74a48f43-d73d-4a88-8ea6-348233c66726-1-0.arrow


CPU times: user 578 ms, sys: 0 ns, total: 578 ms
Wall time: 1.63 s


{'score': 1.5501105410601086,
 'counts': [5043, 2011, 919, 483],
 'totals': [11860, 10634, 9408, 8225],
 'precisions': [42.52107925801012,
  18.911040060184316,
  9.76828231292517,
  5.872340425531915],
 'bp': 0.10577313813540001,
 'sys_len': 11860,
 'ref_len': 38503}

In [23]:
predictions[:10]

['我的项目',
 '如果你的工作不深入,你就不是你的错。',
 '好吧',
 '他说',
 '这是一个伟大的项目',
 '我们知道。',
 '它的意思是',
 '我的意思不是说好吧。',
 '工作3个小时。',
 '不要忘记,他们不懂英语。']

In [24]:
references[:10]

[['而正因为这点，我必须得说， 合作是我最喜欢的超凡能力。 '],
 ['如果你的作品很烂，同样也不全是你的错 '],
 ['（笑）（掌声） 我所讲的就是让人们得到工作， 让孩子们远离街道，让他们了解 自己种植食物的喜悦，荣誉和荣耀， 开放农民市场。 '],
 ['他写道：如果你去看病 医院对你的唯一帮助 就是提供了 给你提供了温暖、食物、住所， 或许还有来自护士的 关怀 '],
 ['接下来，就是我们在使用 社交媒体和手机时最大的教训： '],
 ['我们意识到需要自己走出一条路。 '],
 ['这个名字的意思是“沙漠的边缘”。 '],
 ['我说的不是仅仅是一些告诉你所见是什么的专家们， 而是那些对宇宙各种各样的犄角旮旯都充满激情的人们， 那些分享他们热情的人们 让宇宙成为好客之地。 '],
 ['工作三小时。 '],
 ['不要忘记，她们没学英文。 ']]